In [ ]:
import numpy as np

# Define the matrix of coefficients (7x7 matrix)
A = np.array([
    [1, 1, 1, 1, 1, 1, 1],
    [1, 0, 0, 0, 0, 0, 0],
    [0, 1, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 1, 0],
    [0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 1],
    [1, 1, 0, 0, 0, 0, 0]
])

# Define the constants
c = 0.5  # Example value; set c according to your context
p_a = {
    (0, 0): 0.2,
    (1, 0): 0.3,
    (0, 1): 0.4
}
p_b = {
    (0, 0): 0.2,
    (1, 0): 0.3,
    (0, 1): 0.4
}

# Define the right-hand side vector
b = np.array([
    1 - c,
    p_a[(0, 0)] - c,
    p_a[(1, 0)],
    p_a[(0, 1)],
    p_b[(0, 0)] - c,
    p_b[(1, 0)],
    p_b[(0, 1)]
])

# Solve the linear system A * x = b
try:
    x = np.linalg.solve(A, b)
    print("Solution for [p_{001}, p_{101}, p_{111}, p_{100}, p_{010}, p_{011}, p_{110}]:")
    print(x)
except np.linalg.LinAlgError:
    print("The matrix is singular; no unique solution exists.")